# 🧬 Geometry-Complete Equivariant Diffusion Model
## De Novo Drug Design Training (Everything on Google Drive)

**All code and data stored on Google Drive - persists across sessions!**

## Cell 1: Mount Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
print('✅ Drive mounted')

## Cell 2: Clone Repository to Google Drive

In [ ]:
import os

# Everything goes to Drive
DRIVE_PATH = '/content/drive/MyDrive/geom_diffusion'

if not os.path.exists(DRIVE_PATH):
    print('📥 Cloning repository to Google Drive...')
    !git clone https://github.com/Nethrananda21/geom_diffusion.git {DRIVE_PATH}
else:
    print('✅ Repository already exists on Drive')

%cd {DRIVE_PATH}
!git pull origin master
print(f'\n📁 Working directory: {os.getcwd()}')

## Cell 3: Install Dependencies

In [ ]:
import torch
print(f'GPU: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else "None!"}')

!pip install -q torch-geometric rdkit scipy numpy pyyaml tqdm wandb
print('✅ Dependencies installed')

## Cell 4: Download Dataset to Google Drive

⚠️ Downloads ~50GB. **Skip if already downloaded before.**

In [ ]:
import os

%cd /content/drive/MyDrive/geom_diffusion

RAW_DATA = 'data/CrossDocked2020'

if os.path.exists(RAW_DATA):
    print(f'✅ Dataset already exists: {RAW_DATA}')
    !du -sh {RAW_DATA}
else:
    print('📥 Downloading CrossDocked2020 to Google Drive...')
    print('   Takes 30-60 min. Progress bar shows status.')
    !mkdir -p data
    !curl -L --progress-bar http://bits.csb.pitt.edu/files/crossdock2020/CrossDocked2020_v1.3.tgz | tar -xzf - -C data/
    print('\n✅ Download complete!')

## Cell 5: Preprocess Dataset

In [ ]:
import os
from pathlib import Path

%cd /content/drive/MyDrive/geom_diffusion

train_pkl = Path('data/crossdocked/train_data.pkl')
if train_pkl.exists():
    print(f'✅ Already preprocessed: {train_pkl}')
else:
    raw = Path('data/CrossDocked2020')
    if raw.exists():
        print('⏳ Preprocessing (10-20 min)...')
        !python preprocess_crossdocked.py \
            --data_dir data/CrossDocked2020 \
            --output_dir data/crossdocked \
            --config configs/debug_t4.yaml
        print('\n✅ Done!')
    else:
        print('❌ Run Cell 4 first to download data')

## Cell 6: Delete Cache & Verify

In [ ]:
import shutil
from pathlib import Path

%cd /content/drive/MyDrive/geom_diffusion

cache = Path('data/cache')
if cache.exists():
    shutil.rmtree(cache)
    print('🗑️ Deleted cache')

train = Path('data/crossdocked/train_data.pkl')
val = Path('data/crossdocked/val_data.pkl')
if train.exists() and val.exists():
    print(f'✅ Ready! Train: {train.stat().st_size/1e6:.1f}MB, Val: {val.stat().st_size/1e6:.1f}MB')
else:
    print('⚠️ Using synthetic data')

## Cell 7: Start Training 🚀

In [ ]:
%cd /content/drive/MyDrive/geom_diffusion
!python train.py --config configs/debug_t4.yaml --checkpoint_dir checkpoints

## Cell 8: Resume Training (After Reconnect)

In [ ]:
# After reconnecting: Run Cell 1 (mount), then this cell
# %cd /content/drive/MyDrive/geom_diffusion
# !python train.py --config configs/debug_t4.yaml --resume checkpoints/best_model.pt

## Cell 9: Check Checkpoints

In [ ]:
!ls -la /content/drive/MyDrive/geom_diffusion/checkpoints/